# **Introduction**

This notebook serves as a method of testing the creation of parameterized MuJoCo environments programatically.

# **Imports**

This section imports the necessary packages.

In [283]:
# imports:
import mujoco as mj
import mujoco.viewer
import numpy as np
import json

# **Define Functions**

This section defines the main object-oriented approach taken to create the environment.

In [284]:
# define main class for creating the environment:
class MakeEnv:
    """
    this class is for creating environments pythonically using the python API for mujoco 
    
    """
    # constructor:
    def __init__(self, 
                 params : dict):
        """ 
        this is the constructor for the class, which does the instantiation of the environment

        params:     a dict that contains the relevant parameters for creating the environment

        """
        ### OBJECT PARAMETERS ###
        # env settings:
        self.env_name = params["env_settings"]["name"]

        # compiler settings:
        self.compiler_angle = params["compiler_settings"]["compiler_angle"]

        # option settings:
        self.timestep = params["option_settings"]["timestep"]
        self.integrator = params["option_settings"]["integrator"]
        self.gravity = params["option_settings"]["gravity"]

        # default settings:
        self.joint_damping = params["default_settings"]["joint_damping"]

        # visual settings:
        self.znear = params["visual_settings"]["znear"]
        self.zfar = params["visual_settings"]["zfar"]
        self.shadowsize = params["visual_settings"]["shadowsize"]
        self.framelength = params["visual_settings"]["framelength"]
        self.framewidth = params["visual_settings"]["framewidth"]
        self.jointlength = params["visual_settings"]["jointlength"]
        self.jointwidth = params["visual_settings"]["jointwidth"]

        # skybox settings:
        self.skybox_name = params["skybox_settings"]["name"]
        self.skybox_type = mj.mjtTexture.mjTEXTURE_SKYBOX
        self.skybox_builtin = mj.mjtBuiltin.mjBUILTIN_GRADIENT
        self.skybox_rgb1 = params["skybox_settings"]["rgb1"]
        self.skybox_rgb2 = params["skybox_settings"]["rgb2"]
        self.skybox_width = params["skybox_settings"]["width"]
        self.skybox_height = params["skybox_settings"]["height"]

        # light settings:
        self.light_name = params["light_settings"]["name"]
        self.light_pos = params["light_settings"]["pos"]
        self.light_diffuse = params["light_settings"]["diffuse"]
        self.light_specular = params["light_settings"]["specular"]
        self.light_ambient = params["light_settings"]["ambient"]

        # camera settings:
        self.camera_name = params["camera_settings"]["name"]
        self.camera_pos = params["camera_settings"]["pos"]

        # wall settings:
        self.wall_type = mj.mjtGeom.mjGEOM_BOX
        self.wall_contype = params["wall_settings"]["contype"]
        self.wall_conaffinity = params["wall_settings"]["conaffinity"]
        self.wall_thickness = params["wall_settings"]["thickness"]
        self.wall_height = params["wall_settings"]["height"]
        self.wall_indices = [("right", [1, 0, 0]),
                             ("left", [-1, 0, 0]),
                             ("front", [0, 1, 0]),
                             ("back", [0, -1, 0])]
        
        # ground plane settings:
        self.ground_name = params["ground_settings"]["name"]
        self.ground_type = mj.mjtGeom.mjGEOM_PLANE
        self.ground_contype = params["ground_settings"]["contype"]
        self.ground_conaffinity = params["ground_settings"]["conaffinity"]
        self.ground_internal_length = params["ground_settings"]["internal_length"]
        self.ground_actual_length = self.ground_internal_length + 2 * self.wall_thickness
        self.ground_z_spacing = params["ground_settings"]["z_spacing"]
        self.ground_size = [self.ground_actual_length, self.ground_actual_length, self.ground_z_spacing]
        self.ground_pos = params["ground_settings"]["pos"]
        self.ground_rgba = params["ground_settings"]["rgba"]

    # function for initializing the MjSpec:
    def make_spec(self):
        """ 
        this function initializes the MjSpec and applies the passed basic settings/requirements for the 
        environment (plane, skybox, light, camera, walls, etc.)

        """
        # initialize spec:
        self.spec = mj.MjSpec()

        # set the compiler settings:
        self.spec.compiler.degree = self.compiler_angle

        # set the option settings:
        self.spec.option.timestep = self.timestep
        self.spec.option.integrator = self.integrator
        self.spec.option.gravity = self.gravity
        
        # set the visualization settings:
        self.spec.visual.quality.shadowsize = self.shadowsize
        self.spec.visual.map.znear = self.znear
        self.spec.visual.map.zfar = self.zfar
        self.spec.visual.scale.framelength = self.framelength
        self.spec.visual.scale.framewidth = self.framewidth
        self.spec.visual.scale.jointlength = self.jointlength
        self.spec.visual.scale.jointwidth = self.jointwidth

        # set the default settings:
        self.spec.default.joint.damping = self.joint_damping

        # add the skybox:
        self.spec.add_texture(name = self.skybox_name,
                              type = self.skybox_type,
                              builtin = self.skybox_builtin,
                              width = self.skybox_width, 
                              height = self.skybox_height, 
                              rgb1 = self.skybox_rgb1,
                              rgb2 = self.skybox_rgb2)
        
        # add the light:
        self.spec.worldbody.add_light(name = self.light_name,
                                 pos = self.light_pos, 
                                 diffuse = self.light_diffuse,
                                 specular = self.light_specular, 
                                 ambient = self.light_ambient)
        
        # add camera:
        self.spec.worldbody.add_camera(name = self.camera_name,
                                       pos = self.camera_pos)
        
        # add ground plane:
        self.spec.worldbody.add_geom(name = self.ground_name,
                                     type = self.ground_type,
                                     contype = self.ground_contype,
                                     conaffinity = self.ground_conaffinity,
                                     pos = self.ground_pos,
                                     size = self.ground_size, 
                                     rgba = self.ground_rgba)
        
        # add walls:
        for name, axis in self.wall_indices:
            # if its an x-wall:
            if abs(axis[0]):
                wall_size = [self.wall_thickness, self.ground_actual_length, self.wall_height]
                wall_pos = [axis[0] * (self.ground_actual_length - self.wall_thickness), 0, self.wall_height]
            # else its a y-wall:
            else:
                wall_size = [self.ground_actual_length - 2 * self.wall_thickness, self.wall_thickness, self.wall_height]
                wall_pos = [0, axis[1] * (self.ground_actual_length - self.wall_thickness), self.wall_height]

            # add the geom to the spec
            self.spec.worldbody.add_geom(name = name,
                                        type = self.wall_type,
                                        contype = self.wall_contype,
                                        conaffinity = self.wall_conaffinity,
                                        pos = wall_pos,
                                        size = wall_size)

    # function for adding in an agent:
    def add_agent(self, agent_pos):
        """ 
        this function spawns an agent in the environment. the structure of the agent is subject to change due to the 
        difficulty in insuring the agent moves correctly
        
        """
        agent_base = self.spec.worldbody.add_body(name = "agent_base", pos = agent_pos)
        agent_base.add_joint(name = "agent_x_slide", type = mj.mjtJoint.mjJNT_SLIDE, axis = [1, 0, 0])
        agent_base.add_joint(name = "agent_y_slide", type = mj.mjtJoint.mjJNT_SLIDE, axis = [0, 1, 0])
        agent_base.add_geom(name = "agent_base", type = mj.mjtGeom.mjGEOM_CYLINDER, size = [0.1, 0.01, 0], contype = 0, conaffinity = 0, rgba = [0, 0, 0, 0])

        agent_yaw = agent_base.add_body(name = "agent_yaw", pos = [0, 0, 0])
        agent_yaw.add_joint(name = "agent_z_yaw", type = mj.mjtJoint.mjJNT_HINGE, axis = [0, 0, 1])
        agent_yaw.add_geom(name = "agent_body", type = mj.mjtGeom.mjGEOM_CYLINDER, size = [0.1, 0.01, 0], contype = 1, conaffinity = 1, mass = 0.5, rgba = [1, 0, 0, 1])

        # add the actuators:
        self.spec.add_actuator(name = "x_translate", trntype = mj.mjtTrn.mjTRN_JOINT, target = "agent_x_slide", ctrlrange = [-0.25, 0.25], gear = [0.05, 0, 0, 0, 0, 0])
        self.spec.add_actuator(name = "y_translate", trntype = mj.mjtTrn.mjTRN_JOINT, target = "agent_y_slide", ctrlrange = [-0.25, 0.25], gear = [0.05, 0, 0, 0, 0, 0])
        self.spec.add_actuator(name = "z_rotation", trntype = mj.mjtTrn.mjTRN_JOINT, target = "agent_z_yaw", ctrlrange = [-0.25, 0.25], gear = [0.05, 0, 0, 0, 0, 0])

    # function for adding in a task:
    def add_task(self, task_pos):
        """
        this function spawns a task in the environment
        
        """
        task = self.spec.worldbody.add_body(name = "goal", pos = task_pos)
        task.add_geom(name = "goal", type = mj.mjtGeom.mjGEOM_CYLINDER, size = [0.1, 0.01, 0], contype = 0, conaffinity = 0, rgba = [0, 1, 0, 1])

    # function for compiling the model:
    def compile(self):
        """ 
        this function compiles the model

        """
        self.model = self.spec.compile()

    # not sure if a recompile function is needed, that is the rationale behind splitting up make_spec and compile

    # function for making the environment:
    def make_env(self, agent_pos, task_pos):
        """ 
        this function uses the methods above and basically just chains them together to make and compile the environment
        
        """
        # initialize the spec:
        self.make_spec()

        # add the agent:
        self.add_agent(agent_pos = agent_pos)

        # add the task:
        self.add_task(task_pos = task_pos)

        # compile into model:
        self.compile()

    # function for rendering the environment:
    def render(self):
        """ 
        this function renders and steps through the environment every timestep
        
        """
        # get model data:
        self.data = mj.MjData(self.model)

        # launch a passive window using the model and the data contained within:
        with mujoco.viewer.launch_passive(self.model, self.data) as self.viewer:
            # switch the camera:
            self.viewer.cam.type = mj.mjtCamera.mjCAMERA_FIXED
            self.viewer.cam.fixedcamid = self.model.camera(self.camera_name).id

            # enable viewer options:
            self.viewer.opt.frame = mj.mjtFrame.mjFRAME_BODY
            self.viewer.opt.flags[mj.mjtVisFlag.mjVIS_JOINT] = True
            
            # while viewer is active, step the model every timestep:
            while self.viewer.is_running():
                mujoco.mj_step(self.model, self.data)
                self.viewer.sync()

# **Define Parameters**

This section defines the parameters for use in the creation of the MuJoCo environment.

In [285]:
# open the JSON file:
with open("environment_params.json") as f:
    params = json.load(f)

# **Instantiate and Render**

This section instantiates the model and renders the simulation.

In [288]:
# make environment from class:
environment = MakeEnv(params)
environment.make_env(agent_pos = [0, 0, 0.01], task_pos = [0.5, 0.5, 0.01])
environment.render()